In [ ]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

### Make scene

In [ ]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.panda, None,
                INDY_IP)]
              , connection_list=[False])
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# # deprecated: s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"panda0": ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

## Preapare background

In [ ]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
# wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.3,0,0), 
#                            rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="panda0_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
grip0 = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
gscene.show_pose(crob.home_pose)

## Door

In [ ]:
from pkg.utils.code_scraps import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

In [ ]:
door_s, hinge_s = add_door(pscene, center=(0.7,-0.2,0.5), color=(0.8,0.4,0.2,1))
knob = gscene.create_safe(GEOTYPE.BOX, 'knob', link_name="base_link",
                          center=(-door_s.geometry.dims[0]/2-0.01-0.001,-door_s.geometry.dims[1]/2+0.05,0), 
                          rpy=(0,-np.pi/2,0), dims=(0.02,0.02,0.02),
                          fixed=False, collision=True, color=(0.8,0.8,0.8,1), parent="door")

lever_s, knob_s = add_lever(pscene, knob, door_s, dims=(0.02,0.1,0.02), knob_offset=0.04)

### prepare planner

In [ ]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

In [ ]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene, node_trial_max=30)
tplan.prepare()

ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

In [ ]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
mplan.motion_filters = [GraspChecker(pscene), ReachChecker(pscene)]
mplan.incremental_constraint_motion = True
def explicit_rule(pscene, node, leaf):
    if node[-2:] == ("r2", "knob"):
            return False
    if node[-2] != leaf[-2] and node[-1] != "grip0":
            return False
    if node[-1]=="grip0" and leaf[-1]=="knob" and node[-2]!="r1":
            return False
    if node[1] != leaf[1] and node[0] != "lever":
            return False
    if node[0] == "lever" and node[-2] != leaf[-2]:
            return False
    if node[:2] != leaf[:2] and node[-2]!="r2":
            return False
    return True
tplan.explicit_rule = explicit_rule

### Plan

In [ ]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)

In [ ]:
goal_nodes = [('door_frame', 'h1', 'r1', 'knob')]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=100, 
              multiprocess=True, add_homing=True)

In [ ]:
pscene.set_object_state(initial_state)
gscene.show_pose(initial_state.Q)

In [ ]:
snode_schedule = tplan.get_best_schedule(at_home=True)
ppline.play_schedule(snode_schedule)

### push back

In [ ]:
term_state = snode_schedule[-1].state
pscene.set_object_state(term_state)
gscene.show_pose(term_state.Q)
print(pscene.subject_name_list)
print(term_state.node)

In [ ]:
goal_nodes = [('door_frame', 'h0', 'r1', 'knob')]
ppline.search(term_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

In [ ]:
pscene.set_object_state(term_state)
gscene.show_pose(term_state.Q)

In [ ]:
snode_schedule = tplan.get_best_schedule(at_home=True)
ppline.play_schedule(snode_schedule)